# Example Use of Dark-MaRK Package

The Dark Mark package is designed to create bespoke predictions for the direct detection of dark matter. It offers users the ability to alter the halo model, detector, and dark matter candidate. It also allows users to calculate their differential event rate using either analytical models, or numeric integration techniques using input simulations. 


This tutorial will demonstrate how to use Dark MaRK to test the expected signal of a dark matter direct detector, given an example simulation. Further documentation can be found at https://dark-mark.readthedocs.io/en/latest/. To ensure this example notebook can run, there are some additional requirements in requirements_dev.txt. 

In [2]:
!pip install -r ../requirements_dev.txt

#### Import neccessary packages

In [29]:
import astropy.units as u 
import astropy.constants as const  
import darkmark
from darkmark_plotting import vdf_load_plot, spectral_func_plot, annual_modulation_plot #import plotting scripts for display purposes
from IPython.display import IFrame
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,mark_inset)
import numpy as np 
import numexpr as ne
import os
import scipy.stats as stats
maxwell = stats.maxwell
                                                  
dru = 1./(u.d*u.kg*u.keV)
tru = 1./(u.d*u.kg)

#### Import some example data - in this case we are downloading a Milky Way halo analogue from the FIRE collaborations latte suite. They are hosted on [this website](https://girder.hub.yt/#collection/5b0427b2e9914800018237da) with further information [here](https://fire.northwestern.edu/milky-way/). We download only 1/4 of the particles due to large file size.  The file is large so expect a short wait for download. The download file is then named accordingly.

In [2]:
!wget https://girder.hub.yt/api/v1/item/5b33f745323d120001bfe48d/download

--2021-10-18 16:38:13--  https://girder.hub.yt/api/v1/item/5b33f745323d120001bfe48d/download
Resolving girder.hub.yt (girder.hub.yt)... 141.142.231.2
Connecting to girder.hub.yt (girder.hub.yt)|141.142.231.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2460753960 (2.3G) [application/octet-stream]
Saving to: ‘download’

download            100%[===================>]   2.29G  4.02MB/s    in 20m 43s 

2021-10-18 16:58:57 (1.89 MB/s) - ‘download’ saved [2460753960/2460753960]



In [6]:
!mv download snapshot_600.1.hdf5

#### Load in the functions from the darkmark package that are needed

In [1]:
from darkmark.boostrap import bootstrap_rate
from darkmark.simload import load_sim_profile 
from darkmark.sampling_transformations import def_samp_coord, calc_real_uE_pE,sample_dm_solcirc 
from darkmark.dm_detector_class import Nibbler 

/Users/gracelawrence/Documents/2021/Dark-MaRK


#### Define the parameters of the experiment you want to model. Some of these parameters are not called in the example functions below.

In [2]:
v0 = 233 #Average Velocity Speed of Dark Matter in the Halo in km/s.
rho_dm = 0.3 #Dark Matter Density in (Gev/c^2)/cm^3.
M_dm = 60 #Dark Matter Particle Mass in GeV/c^2.
A = 73 #Atomic Mass of the Target Detector Nuclei.
sigma_0 = 1.3e-41*u.cm**2 #Cross-section.
v_max = 600 #Escape velocity of the system in km/s.
period = 365 #The time period to perform the calculates over (typically 1 year) in units of days.
energy_range = np.linspace(0,100,100) #The energy region of interest.
detvar =('Ge') #The detector element to investigate - Germanium.
dmvar=('CDM') #The Dark Matter cosmology assumed (CDM is currently only option with WDM and SiDM in development).
vdf_var =('SHM') #The velocity model to use, 'SHM' (standard halo model) is currently the default.

#### Define the Nibbler class 

In [ ]:
nib = Nibbler(detvar, dmvar, 'Nucleon', vdf_var, v0,rho_dm, M_dm, sigma_0, v_max,period)                                           

#### Define the paths to the simulation data, and to the output folder

In [3]:
file_path = "./snapshot_600.1.hdf5" 

In [4]:
results_path = "./DarkMark_Example_Outputs/"
CHECK_FOLDER = os.path.isdir(results_path)
if not CHECK_FOLDER:
    os.makedirs(results_path)

#### Define how many evenly spaced samples you'd like to take around the solar circle.

In [5]:
n_samples = 8 

#### Load the simulation profile 

#### Load the simulation, centre and align it, and then visualize the dark matter component.

In [10]:
target,s = load_sim_profile(True,file_path,True, True, results_path, cosmology = 'WMAP1') 

In [20]:
IFrame("./DarkMark_Example_Outputs/DarkMaRK_results/DM_visualization.png", width=500, height=500)

#### Generate the co-ordinates at which to samples the dark matter

In [8]:
sample_coords, theta_ang = def_samp_coord(target, n_samples) 

#### Investigate the sampling co-ordinates positions

In [9]:
print(sample_coords)
plt.clf()
plt.scatter(sample_coords[:,0],sample_coords[:,1])
plt.xlabel('x (kpc)')
plt.ylabel('y (kpc)')
plt.show()

#### Sample the dark matter and generate the velocity disitribution function in both the galactocentric and geocentric reference frames, for each of the samples.

In [11]:
vel_galacto, vel_geo = sample_dm_solcirc(nib,sample_coords, target, theta_ang, 'milkyway',results_path, False)   

#### Load and display the geocentric versus galactocentric velocity speed distributions for a given day of the year, for a given sample around the solar circle.

In [12]:
day = 152
sample = 5
vdf_load_plot(day,results_path, sample)

#### Integrate over the velocity distributions to generate spectral rate functions.

In [13]:
bootstrap_rate(nib, n_samples, results_path, False) 

In [22]:
spectral_func_plot(nib, sample, results_path) 

#### Integrate the spectral rate function, over a given energy band, to generate an annual modulation prediction. 

Here we integrate the spectral rate function of sample 6, between energies 2 and 6keV. 

In [23]:
annual_modulation_plot(nib, 6, 2 , 6, 'Nuclear', results_path) 

Optionally, integrate over electron equivalent (quenched) energies, kevee.

In [24]:
annual_modulation_plot(nib, 6, 2 , 6, 'E_Equiv', results_path) 